# 民主主義ー権威主義ボードゲーム

### 目次
1. [概要](#sec1)
2. [ルール説明](#sec2_1)
    1. [手順](#sec2_2)
    2. [クーデタ](#sec2_3)
    3. [権威主義国の裏切り](#sec2_4)
    4. [財政破綻](#sec2_5)
    5. [民主化支援](#sec2_6)
3. [参考](#sec3)
3. [コード](#sec4)

## 1. 概要<a id= "sec1"></a>
- 基本的な動作はオセロ（リバーシボード）と同じである。
- オセロの一つ一つの石を国、ボード全体を国際社会に見立てて、民主主義サイド　対　権威主義サイドで相手サイドよりも多くの国を自分の体制にすることを目指す。（オセロの石の黒＝権威主義、白＝民主主義というイメージである）
- オセロと異なり４つの新ルール(`クーデタ` `財政破綻` `権威主義の裏切り` `民主化支援`)があることが特徴である。
- 新ルールの内容とその国際政治における背景を説明する

## 2. ルール説明<a id= "sec2_1"></a>

### A. 手順<a id= "sec2_2"></a>
- ①プレイヤー名を入力する
- ②民主主義サイド、権威主義サイドに分かれたプレイヤーが交互に自分の国を作っていく。最初は民主主義サイドから始める
- ③国を作ることのできるマスの座標が表示されるので、国を作りたいマスのインデックスを入力する。（インデックスは0から始まる）
- ④通常のオセロとは異なり`クーデタ`(５の倍数）、`財政破綻`(７の倍数）、`権威主義の裏切り`(８の倍数）、`民主化支援`（１５の倍数）が起こる
- ⑤どちらのプレイヤーも新しく国を作れないときゲームが終了する。ゲーム終了時に勝者（同数の場合はドロー）とそれぞれの国の数が表示される

- 国を作れないときはパスされる
- インデックスに`-1`を入力するとゲームを強制終了できる

### B. クーデタ<a id="sec2_3"></a>
- 5の倍数回のターン後に、民主主義の国（1）がランダムに権威主義に変わる。
- 多くの民主主義国にとって、（特に冷戦前は）軍のクーデタが最大の脅威だった。

### C. 権威主義国の裏切り<a id="sec2_4"></a>
- 8の倍数回の権威主義のターンで、新たに作られた権威主義の国は、権威主義サイドを裏切り民主主義サイドの味方をする。
- このターンでは、新生国と民主主義国に挟まれた権威主義国が民主主義国に変わってしまう

- 権威主義国はお互いの仲が悪くなると戦争が起こることがあるが、民主主義国同士の間では戦争が起こる確率は極めて低い、と言われている。これを`「民主的平和論」`と呼ぶ。

### D. 財政破綻<a id="sec2_5"></a>
- 7の倍数回のターン後に、ランダムで国（1）が「破綻国」に変わる。
- 破綻国は「民主支援」がない限り、権威主義にも民主主義にもならない。また自分の国で相手の国を挟んでもその中に破綻国がある場合は、相手の国を自分の国に変えることはできない
- 財政破綻を起こすなどして機能しなくなった国家を`「破綻国家」`という

### E. 民主化支援<a id="sec2_6"></a>
- 15倍数回のターン終了時に、民主側は破綻国を一つ選び民主国に変えることができる。
- 冷戦後、欧米諸国を中心に、民主制をすることを条件に財政支援を行った。これを一般的に`「民主化支援」`と呼ぶ。

## 3. 参考<a id="sec3"></a>
- マスの探索方法などは、`katoh4h`さんのブログを参考にさせていただいた。非常に直感的に分かるコードの表記方法だった（https://katoh4u.hatenablog.com/entry/2018/03/22/130105）
- 今回はアルゴリズムの勉強を重視してGUIツールを用いずに創作したが、やはりビジュアルが非常に見にくいという問題があった。今後はGUIツールを用いてこのゲームを改良していきたい。

## 4. コード<a id="sec4"></a>

In [ ]:
import random

Democracy = -1
Failed = -2
Autocracy = -3
Draw = -4
BOARD_SIZE = 8

In [ ]:
class Board:

    """ ボードを作る。マスと国が対応している"""

    def __init__(self):
        self.cells = []
        for i in range(BOARD_SIZE):
            self.cells.append([None for i in range(BOARD_SIZE)])

        self.cells[3][3] = Democracy
        self.cells[3][4] = Autocracy
        self.cells[4][3] = Autocracy
        self.cells[4][4] = Democracy


In [ ]:
class Program:
    
    """戦争ゲームのプログラムを作る。"""
    
    def __init__(self, turn = 1):
        self.board = Board()
        self.turn = turn

    def list_changeable_states(self, x, y, player):
        """国を作ったとき、変更できる国をリストで返す"""
        PREV = -1
        NEXT = 1
        DIRECTION = [PREV, 0, NEXT]
        self.changeable = []

        for dx in DIRECTION:
            for dy in DIRECTION:
                if dx == 0 and dy == 0:
                    continue

                tmp = []
                depth = 0
                while(True):
                    depth += 1
                    rx = x + (dx * depth)
                    ry = y + (dy * depth)
                    #調べる座標(rx, ry)がボードの範囲内ならば
                    if 0 <= rx < BOARD_SIZE and 0 <= ry < BOARD_SIZE:
                        request = self.board.cells[ry][rx]
                        if request is None or request == Failed:
                            break

                        if request == player:
                            if tmp !=[]:
                                self.changeable.extend(tmp)
                            break
                        else:
                            tmp.append((rx, ry))
                    else:
                        break
        return self.changeable

    def list_selectable_states(self, player):
        """国を作ることが可能なマスをリストで返す """
        self.selectable =[]
        for x in range(BOARD_SIZE):
            for y in range(BOARD_SIZE):
                if self.board.cells[y][x] is not None:
                    continue
                if self.list_changeable_states(x, y, player) ==[]:
                    continue
                else:
                    self.selectable.append((x,y))
        if self.selectable ==[]:
            return False
        else:
            return self.selectable

    def make_state(self, x, y, player):
        """　自分の国を新たに作る　"""
        self.list_changeable_states(x, y, player)

        print(self.changeable)

        if self.board.cells[y][x] is not None:
            return False
        elif  self.changeable == []:
            return False

        else:
            self.board.cells[y][x] = player
            return True

    def change_regimes(self, x, y, player):
        """
        相手の国を自分の国に変える。
        新たに作った国と自分の国に挟まれる連続した相手の国を自分の国に変える
        (オセロと同じ)
        """
        for x,y in self.changeable:
            self.board.cells[y][x] = player

    def betray_regimes(self, x, y, player):
        """権威国が味方の国を裏切る。民主国と新生国に挟まれた権威国が民主国に変わる"""
        betrayable = []
        PREV = -1
        NEXT = 1
        DIRECTION = [PREV, 0, NEXT]
        for dx in DIRECTION:
            for dy in DIRECTION:
                if dx == 0 and dy == 0:
                    continue
                tmp = []
                depth = 0
                while(True):
                    depth += 1
                    rx = x + (dx * depth)
                    ry = y + (dy * depth)

                    if 0 <= rx < BOARD_SIZE and 0 <= ry < BOARD_SIZE:
                        request = self.board.cells[ry][rx]
                        if request is None or request == Failed:
                            break
                        elif request == Democracy:
                            if tmp !=[]:
                                betrayable.extend(tmp)
                            break
                        else:
                            tmp.append((rx, ry))
                    else:
                        break
        for x,y in  betrayable:
            self.board.cells[y][x] = Democracy

    def coup_state(self):
        """ランダムで民主国(1)が権威国に転換する"""
        democratic_states = []
        self.couped_state = []
        for y, w in enumerate(self.board.cells):
            for x, v  in enumerate(w):
                if v == Democracy:
                    democratic_states.append((x,y))
        self.couped_state.append(random.choice(democratic_states))
        for x,y in self.couped_state:
            self.board.cells[y][x] = Autocracy

    def fail_state(self):
        """
           財政破綻がおこる。ランダムで国(1)が破綻国になる。
        　破綻国は民主化されるまでは権威国にも民主国にもならない
        """
        states = []
        self.failed_state = []
        for y, w in enumerate(self.board.cells):
            for x, v  in enumerate(w):
                if v == Autocracy or v == Democracy:
                    states.append((x,y))
        self.failed_state.append(random.choice(states))
        for x,y in self.failed_state:
            self.board.cells[y][x] = Failed

    def democratize_state(self):
        """
        民主支援。破綻国(1)が民主国になる。対象の破綻国は民主プレーヤーが選べる
        """
        self.can_democratize_states = []
        for x in range(BOARD_SIZE):
            for y in range(BOARD_SIZE):
                if self.board.cells[y][x] == Failed:
                    self.can_democratize_states.append((x,y))
        return self.can_democratize_states




In [ ]:
class Game(Program):
    
    def __init__(self ,start_player = Democracy, start_states = {"Democracy": 0, "Autocracy": 0}):
        super().__init__()
        self.player = start_player
        self.board = Board()
        self.winner = None
        self.was_passed = False
        self.states = start_states
        self.player_democracy = str(input("PlayerName_Democracy: "))
        self.player_autocracy = str(input("PlayerName_Autocracy: "))

    def get_current_player(self):
        return self.player

    def get_next_player(self):
        if self.player == Autocracy:
            return Democracy
        else:
            return Autocracy

    def shift_player(self):
        self.player = self.get_next_player()

    def get_player_name(self, player):
        """プレイヤー名と体制を結びつける"""
        if player == Democracy:
            return self.player_democracy
        elif player == Autocracy:
            return self.player_autocracy
        else:
            False


    def list_selectable_states(self):
        return super().list_selectable_states(self.player)

    def make_state(self, x, y):
        if super().make_state(x, y, self.player):
            if self.turn%8 == 0:
                self.betray_regimes(x, y , self.player)
                print("Autocracy betrays")
            else:
                self.change_regimes(x,y, self.player)
            self.was_passed = False
            self.shift_player()
            self.turn += 1
        else:
            return False

    def coup_state(self):
        if self.turn%5 != 0:
            return False
        else:
            super().coup_state()
            print("next couped_state is " + str(self.couped_state))

    def fail_state(self):
        if self.turn%7 != 0:
            return False
        else:
            super().fail_state()
            print("next failed_state is " + str(self.failed_state))

    def democratize_state(self):
        if self.turn%15 != 0:
            return False
        else:
            democratized_state=[]
            super().democratize_state()
            print("you can democratize :" + str(self.can_democratize_states))
            self.democratize_index = int(input("democratize :"))
            democratized_state.append(self.can_democratize_states[self.democratize_index])
            for x, y in democratized_state:
                self.board.cells[y][x] = Democracy


    def choose_cell(self):
        """新生国のマスをリストのインデックスで選択する"""
        self.list_selectable_states()
        while True:
            self.index = int(input("choose : "))
            if self.index == -1:
                return False
                break
            if self.list_selectable_states() is False:
                return False
                break
            if self.index not in list(range(len(self.selectable))):
                print("Please choose the proper index")
                continue
            else:
                break
        self.make_state(*self.selectable[self.index])

    def pass_turn(self):
        """選択可能なマスがないときターンをパスをする"""
        if self.list_selectable_states() is not False:
            return False
        else:
            self.was_passed = True
            print("player {} can not puts.". format(self.get_player_name(self.player)))
            self.shift_player()

    def count_stones(self):
        """終局したとき、お互いの国の数を数える"""
        for x in range(BOARD_SIZE):
            for y in range(BOARD_SIZE):
                if self.board.cells[x][y] == Democracy:
                    self.states["Democracy"] +=1
                if self.board.cells[x][y] == Autocracy:
                    self.states["Autocracy"] += 1

    def is_finished(self):
        if self.was_passed == True:
            if self.list_selectable_states() is False:
                return True
            else:
                False
        else:
            False

    def draw_result(self):
        """ゲームが終了したとき処理して結果を返す"""
        self.count_stones()
        if self.states["Democracy"] < self.states["Autocracy"]:
            self.winner = Autocracy
        elif self.states["Autocracy"] < self.states["Democracy"]:
            self.winner = Democracy
        else:
            self.winner = Draw


    def exit_game(self):
        """インデックス選択で-1を入力したらゲームを強制終了する"""
        if self.index == -1:
            return True

    def show_board(self):
        """ボードを表示する"""
        print("--"*20)
        for i in self.board.cells:
            for cell in i:
                if cell == Democracy:
                    print("D", end ="")
                elif cell == Autocracy:
                    print("A", end ="")
                elif cell == Failed:
                    print("F" , end = "")
                else:
                    print("#", end = "")
            print("\n", end = "")

    def show_items(self):
        """アイテムを表示する"""
        print("player:" + self.get_player_name(self.get_current_player()))
        print("put to: "+ str(self.list_selectable_states()))
        print("turn:" + str(self.turn))

    def show_finished(self):
        """終了画面を表示する"""
        if self.winner == Draw:
            print("Draw")
        else:
            print("Winner is " + self.get_player_name(self.winner))
        print("{}:{}".format("DEMOCRACY",self.states["Democracy"]))
        print("{}:{}".format("AUTOCRACY", self.states["Autocracy"]))

    def play_game(self):
        """機能を統合してゲームを完成する"""
        while(True):
            if game.is_finished():
                game.draw_result()
                game.show_board()
                game.show_finished()
                break

            if game.pass_turn():
                game.shift_player()
                continue


            game.show_board()
            game.show_items()

            game.choose_cell()
            if game.exit_game():
                break

            game.coup_state()
            game.fail_state()
            game.democratize_state()




In [ ]:
if __name__ == "__main__":
    game = Game()
    game.play_game()
